In [44]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [45]:
load_dotenv()

True

In [46]:
GROQ_API_TOKEN = os.getenv("GROQ_API_TOKEN")

In [47]:
os.environ["GROQ_API_TOKEN"] = GROQ_API_TOKEN

In [48]:
with open("Response.json", "r") as file:
    RESPONSE_JSON = json.load(file)

In [49]:
print(RESPONSE_JSON)

{'1': {'no': 1, 'question': 'multiple choice question', 'options': {'a': 'option a', 'b': 'option b', 'c': 'option c', 'd': 'option d'}, 'answer': 'a'}, '2': {'no': 2, 'question': 'multiple choice question', 'options': {'a': 'option a', 'b': 'option b', 'c': 'option c', 'd': 'option d'}, 'answer': 'a'}, '3': {'no': 3, 'question': 'multiple choice question', 'options': {'a': 'option a', 'b': 'option b', 'c': 'option c', 'd': 'option d'}, 'answer': 'a'}}


In [87]:
prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template="""
    Text: {text}
    You are an expert MCQ maker. Given the above text, it is you job to \
    create a quiz of {number} multiple choice questions (MCQs) with answers for {subject} students in {tone} tone.
    Make sure the questions are not repeated. check all the questions to be conforming text as well.
    Make sure to format your response in JSON format with the following structure: ad use it as a guide:
    Ensure to make {number} MCQs
    {RESPONSE_JSON}
    """
)

In [88]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=GROQ_API_TOKEN,
    temperature=0.7
)

In [89]:
chain = LLMChain(llm=llm, prompt=prompt, output_key='quiz', verbose=True)

In [90]:
template = """
    Yor are an expert english grammarian and writer. Given a multiple choice quiz for {subject} students, it is your job to \
    You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use max 50 words.
    if the quiz is not at per with the cognitive and analytical abilities of the students, \
    update the quiz questions which need to be changed and change the tone such that is perfectly fits the students abilities.
    Quiz_MCQs: {quiz}
    Check from an expert english Writer of the above quiz:
    """

In [91]:
prompt2 = PromptTemplate(
    input_variables=["quiz", "subject"],
    template=template
)

In [92]:
review_chain = LLMChain(llm=llm, prompt=prompt2, output_key='review', verbose=True)

In [117]:
generate_evaluate_chain = SequentialChain(
    chains=[chain, review_chain],
    input_variables=['text', 'number', 'subject', 'tone', 'RESPONSE_JSON'],
    output_variables=['quiz', 'review'],
    verbose=True)

In [118]:
with open ("text.txt", "r") as file:
    text = file.read()

In [119]:
print(text)

Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora. Technology companies developing generative AI include OpenAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.

Generative AI has raised many ethical questions as it can be used for cyberc

In [120]:
text
NUMBER = 5
SUBJECT = "AI"
TONE = "Formal"
RESPONSE_JSON = RESPONSE_JSON

In [126]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain.invoke(
        {
            "text": text,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text: Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora. Technology companies developing generative AI include OpenAI, Anthropic, Meta AI, Micr


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    Yor are an expert english grammarian and writer. Given a multiple choice quiz for AI students, it is your job to     You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use max 50 words.
    if the quiz is not at per with the cognitive and analytical abilities of the students,     update the quiz questions which need to be changed and change the tone such that is perfectly fits the students abilities.
    Quiz_MCQs: Here are 5 multiple choice questions based on the provided text:

```
{
  "1": {
    "no": 1,
    "question": "What is the primary function of generative artificial intelligence (Generative AI)?",
    "options": {
      "a": "To analyze existing data",
      "b": "To produce text, images, videos, or other forms of data based on input",
      "c": "To develop new algorithms",
      "d": "To improve cybersecurity"
    },
    "answer": "b"
  },
  

In [127]:
print(f"Total Tokens Used: {cb.total_tokens}")
print(f"Total Cost: ${cb.total_cost:.4f}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")

Total Tokens Used: 2037
Total Cost: $0.0000
Prompt Tokens: 1507
Completion Tokens: 530


In [128]:
response

{'text': 'Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.\n\nGenerative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora. Technology companies developing generative AI include OpenAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.\n\nGenerative AI has raised many ethical questions as it can be u

In [129]:
quiz=response.get('quiz')
print(quiz)
print(type(quiz))

Here are 5 multiple choice questions based on the provided text:

```
{
  "1": {
    "no": 1,
    "question": "What is the primary function of generative artificial intelligence (Generative AI)?",
    "options": {
      "a": "To analyze existing data",
      "b": "To produce text, images, videos, or other forms of data based on input",
      "c": "To develop new algorithms",
      "d": "To improve cybersecurity"
    },
    "answer": "b"
  },
  "2": {
    "no": 2,
    "question": "What technological advancement in the 2020s contributed to the development of generative AI tools?",
    "options": {
      "a": "Improvements in rule-based expert systems",
      "b": "Advances in computer vision",
      "c": "Improvements in transformer-based deep neural networks, particularly large language models (LLMs)",
      "d": "Development of new programming languages"
    },
    "answer": "c"
  },
  "3": {
    "no": 3,
    "question": "What is an example of an early algorithm used for generating med

In [130]:
import json
import re

# Assuming `quiz` is the response string from your LLM
# Step 1: Extract JSON inside triple backticks using regex
match = re.search(r"```(.*?)```", quiz, re.DOTALL)

if match:
    json_str = match.group(1).strip()  # Clean and extract JSON block
    try:
        quiz_dict = json.loads(json_str)
        print("✅ JSON loaded successfully!")
        print(type(quiz_dict))
        print(quiz_dict)
    except json.JSONDecodeError as e:
        print("❌ JSONDecodeError:", e)
else:
    print("❌ Could not find JSON block in the quiz string.")


✅ JSON loaded successfully!
<class 'dict'>
{'1': {'no': 1, 'question': 'What is the primary function of generative artificial intelligence (Generative AI)?', 'options': {'a': 'To analyze existing data', 'b': 'To produce text, images, videos, or other forms of data based on input', 'c': 'To develop new algorithms', 'd': 'To improve cybersecurity'}, 'answer': 'b'}, '2': {'no': 2, 'question': 'What technological advancement in the 2020s contributed to the development of generative AI tools?', 'options': {'a': 'Improvements in rule-based expert systems', 'b': 'Advances in computer vision', 'c': 'Improvements in transformer-based deep neural networks, particularly large language models (LLMs)', 'd': 'Development of new programming languages'}, 'answer': 'c'}, '3': {'no': 3, 'question': 'What is an example of an early algorithm used for generating media?', 'options': {'a': 'Decision trees', 'b': 'Markov chains', 'c': 'Neural networks', 'd': 'Clustering algorithms'}, 'answer': 'b'}, '4': {'no

In [83]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value['question']
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value['options'].items()
        ]
        )
    correct = value['answer']
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [84]:
quiz_table_data

[{'MCQ': 'What is the primary function of generative artificial intelligence (Generative AI)?',
  'Choices': 'a: To analyze existing data | b: To produce text, images, videos, or other forms of data | c: To optimize machine learning models | d: To develop new algorithms',
  'Correct': 'b'},
 {'MCQ': 'What technological advancement made the development of generative AI tools more common in the 2020s?',
  'Choices': 'a: Improvements in rule-based expert systems | b: Advances in computer vision | c: Improvements in transformer-based deep neural networks, particularly large language models (LLMs) | d: Development of new programming languages',
  'Correct': 'c'},
 {'MCQ': 'What is an example of an early algorithm used for generating media?',
  'Choices': 'a: Markov chain | b: Decision tree | c: Neural network | d: Genetic algorithm',
  'Correct': 'a'},
 {'MCQ': 'What is a potential ethical concern related to the use of generative AI?',
  'Choices': 'a: Job creation | b: Improved data analys

In [85]:
df = pd.DataFrame(quiz_table_data)

In [86]:
df.head()

,MCQ,Choices,Correct
0,What is the primary function of generative art...,a: To analyze existing data | b: To produce te...,b
1,What technological advancement made the develo...,a: Improvements in rule-based expert systems |...,c
2,What is an example of an early algorithm used ...,a: Markov chain | b: Decision tree | c: Neural...,a
3,What is a potential ethical concern related to...,a: Job creation | b: Improved data analysis | ...,c
4,In which decade did Harold Cohen create and ex...,a: 1960s | b: 1970s | c: 1980s | d: 1990s,b


In [ ]:
df.to_csv("generated_quiz.csv", index=False)